In [172]:
import io
import pandas as pd
import requests
import time
import tqdm


def week_dates(date, weekday=0):
    week_start = date - pd.DateOffset(weekday=weekday, weeks=1)
    week_end = date + pd.DateOffset(weekday=weekday, weeks=0)
    return week_start, week_end


def get_chart(date, region='en', freq='daily', chart='top200'):
    chart = 'regional' if chart == 'top200' else 'viral'
    date = pd.to_datetime(date)
    if date.year < 2017:
        raise ValueError('No chart data available from before 2017')
    if freq == 'weekly':
        start, end = week_dates(date, weekday=4)
        date = f'{start.date()}--{end.date()}'
    else:
        date = f'{date.date()}'
    url = f'https://spotifycharts.com/{chart}/{region}/{freq}/{date}/download'
    data = io.StringIO(requests.get(url).text)
    try:
        df = pd.read_csv(data, skiprows=1) # Fix Spotify's Note
    except pd.errors.ParserError:
        df = None
        print(data)
    return df


def get_charts(start, end, region='en', freq='daily', chart='top200', sleep=1):
    sample = 'D' if freq == 'daily' else 'W'
    dfs = []
    for date in tqdm.tqdm(pd.date_range(start=start, end=end, freq=sample)):
        df = get_chart(date, region=region, freq=freq, chart=chart)
        if df is not None:
            df['Date'] = date
            df = df.head(50)
            dfs.append(df)
            time.sleep(sleep)
    return pd.concat(dfs, ignore_index=True)

In [173]:
chart = get_charts('2019-08-21','2019-08-22', freq='daily', region='mx')

100%|██████████| 2/2 [00:03<00:00,  1.60s/it]


In [174]:
chart

,Position,Track Name,Artist,Streams,URL,Date
0,1,China,Anuel AA,724556,https://open.spotify.com/track/2ksOAxtIxY8yElEWw8RhgK,2019-08-21
1,2,Callaita,Bad Bunny,616004,https://open.spotify.com/track/2TH65lNHgvLxCKXM3apjxI,2019-08-21
2,3,LA CANCIÓN,J Balvin,567771,https://open.spotify.com/track/0fea68AdmYNygeTGI4RC18,2019-08-21
3,4,No Me Conoce - Remix,Jhay Cortez,518514,https://open.spotify.com/track/4R8BJggjosTswLxtkw8V7P,2019-08-21
4,5,Tutu,Camilo,511946,https://open.spotify.com/track/1nocRtwyNPVtGcIJqfgdzZ,2019-08-21
5,6,11 PM,Maluma,471825,https://open.spotify.com/track/7KbF6AdprOXEEHlsq11Z6d,2019-08-21
6,7,Soltera (Remix),Lunay,409689,https://open.spotify.com/track/2qMnZF5Ja2WvmfbblByQza,2019-08-21
7,8,Señorita,Shawn Mendes,403161,https://open.spotify.com/track/6v3KW9xbzN5yKLt9YKDYA2,2019-08-21
8,9,Con Altura,ROSALÍA,383658,https://open.spotify.com/track/2qG5sZ7Si6sdK74qLxedYM,2019-08-21
9,10,Otro Trago,Sech,378360,https://open.spotify.com/track/1Ej96GIBCTvgH7tNX1r3qr,2019-08-21


In [176]:
chart.describe()

,Position,Streams
count,100.00000,100.000000
mean,25.50000,266216.340000
std,14.50357,134477.629531
min,1.00000,143027.000000
25%,13.00000,180417.250000
50%,25.50000,213297.500000
75%,38.00000,293761.000000
max,50.00000,724556.000000


In [177]:
chart['Track Name'].unique()

array(['China', 'Callaita', 'LA CANCIÓN', 'No Me Conoce - Remix', 'Tutu',
       '11 PM', 'Soltera (Remix)', 'Señorita', 'Con Altura', 'Otro Trago',
       'QUE PRETENDES', 'Otro Trago - Remix', 'Tra Tra Tra Remix',
       'Loco Contigo (feat. J. Balvin & Tyga)',
       'Baila Conmigo (feat. Kelly Ruíz)', 'Tal Vez', 'Bellacoso', 'HP',
       'Yo x Ti, Tu x Mi', 'Fuentes de Ortiz', 'La Boca - Remix',
       'Desconocidos', 'Runaway', 'Te Vi', 'Amor A Primera Vista',
       'Verte Ir', 'Aleluya', 'Si Supieras', 'Con Calma', 'Chicharrón',
       'Old Town Road - Remix', 'La Boca', 'Lost on You', 'Pa Mí - Remix',
       'How Do You Sleep?', 'Nicki Nicole: Bzrp Music Sessions, Vol. 13',
       'bad guy', 'Calma - Remix', 'Rebota - Remix',
       'Contando Lunares (feat. Cruz Cafuné)', 'Te Robaré',
       'No Lo Trates', "I Don't Care (with Justin Bieber)", 'Secreto',
       'Dancin (feat. Luvli) - Krono Remix', 'Adan y Eva', 'Atrévete',
       'Latina (feat. Maluma)', 'Simplemente Gracias',

In [178]:
chart['Track Id'] = chart['URL'].str.split("/",expand=True)[4]

In [179]:
chart

,Position,Track Name,Artist,Streams,URL,Date,Track Id
0,1,China,Anuel AA,724556,https://open.spotify.com/track/2ksOAxtIxY8yElEWw8RhgK,2019-08-21,2ksOAxtIxY8yElEWw8RhgK
1,2,Callaita,Bad Bunny,616004,https://open.spotify.com/track/2TH65lNHgvLxCKXM3apjxI,2019-08-21,2TH65lNHgvLxCKXM3apjxI
2,3,LA CANCIÓN,J Balvin,567771,https://open.spotify.com/track/0fea68AdmYNygeTGI4RC18,2019-08-21,0fea68AdmYNygeTGI4RC18
3,4,No Me Conoce - Remix,Jhay Cortez,518514,https://open.spotify.com/track/4R8BJggjosTswLxtkw8V7P,2019-08-21,4R8BJggjosTswLxtkw8V7P
4,5,Tutu,Camilo,511946,https://open.spotify.com/track/1nocRtwyNPVtGcIJqfgdzZ,2019-08-21,1nocRtwyNPVtGcIJqfgdzZ
5,6,11 PM,Maluma,471825,https://open.spotify.com/track/7KbF6AdprOXEEHlsq11Z6d,2019-08-21,7KbF6AdprOXEEHlsq11Z6d
6,7,Soltera (Remix),Lunay,409689,https://open.spotify.com/track/2qMnZF5Ja2WvmfbblByQza,2019-08-21,2qMnZF5Ja2WvmfbblByQza
7,8,Señorita,Shawn Mendes,403161,https://open.spotify.com/track/6v3KW9xbzN5yKLt9YKDYA2,2019-08-21,6v3KW9xbzN5yKLt9YKDYA2
8,9,Con Altura,ROSALÍA,383658,https://open.spotify.com/track/2qG5sZ7Si6sdK74qLxedYM,2019-08-21,2qG5sZ7Si6sdK74qLxedYM
9,10,Otro Trago,Sech,378360,https://open.spotify.com/track/1Ej96GIBCTvgH7tNX1r3qr,2019-08-21,1Ej96GIBCTvgH7tNX1r3qr


In [183]:
chart[chart['Artist'] == 'Anuel AA']

,Position,Track Name,Artist,Streams,URL,Date,Track Id
0,1,China,Anuel AA,724556,https://open.spotify.com/track/2ksOAxtIxY8yElEWw8RhgK,2019-08-21,2ksOAxtIxY8yElEWw8RhgK
43,44,Secreto,Anuel AA,160657,https://open.spotify.com/track/5W83ErFkO3aKAIS1WMi6u0,2019-08-21,5W83ErFkO3aKAIS1WMi6u0
50,1,China,Anuel AA,721786,https://open.spotify.com/track/2ksOAxtIxY8yElEWw8RhgK,2019-08-22,2ksOAxtIxY8yElEWw8RhgK
92,43,Secreto,Anuel AA,160056,https://open.spotify.com/track/5W83ErFkO3aKAIS1WMi6u0,2019-08-22,5W83ErFkO3aKAIS1WMi6u0
